In [ ]:
#antihypertensive medication prescribed pre admission to a care home (six months prior) updated social and primary care data 
create or replace table `yhcr-prd-bradfor-bia-core.CB_2172.cb_2172_antihypertensive_pre_admission` as (
  #call all medication for the care home cohort
with care_home_medication as (
  select 
  *,
  from `yhcr-prd-bradfor-bia-core.CB_FDM_PrimaryCare.tbl_srprimarycaremedication`
  where person_id in (select person_id from `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1`)
),
#call bnf code and dmd(snomed) code
e as (select BNF_Code,BNF_Name, cast(SNOMED_CODE as string) as SNOMEDCODE_STRING
  from `yhcr-prd-bradfor-bia-core.CB_LOOKUPS.tbl_BNF_DMD_SNOMED_lkp`),

#map the dmd code in primary care table to the bnf code in lookup table to enable antihypertensives to be identified based on bnf sub sub chapter
f as (select 
d.*,
e.BNF_Code,
e.BNF_Name
from care_home_medication d 
left join e
on d.idmultilexdmd = e.SNOMEDCODE_STRING),

medication_join_admission as (
  select
  f.*,
  c.first_episodestartdate
  from f 
  left join `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` c
  on f.person_id = c.person_id
),
#select antihypertensive medication based on bnf code, 
g as (
select 
SUBSTRING(BNF_code,1,7) as BNF_sub_sub_chapter,
person_id,
datemedicationstart,
datemedicationend,
medicationdosage, 
medicationquantity,
isrepeatmedication,
first_episodestartdate,
BNF_code,
BNF_name,
from medication_join_admission
where (BNF_Code like '0206020%') or (BNF_Code like '0204000%') or (BNF_Code like '0202010%') or (BNF_code like  '020505%') or (BNF_code like '0205040%')
order by person_id),
  # select repeat medication prescribed six months prior to care home admission
mpad_1 as (
select
*
from g
where datemedicationstart between datetime_sub(first_episodestartdate, INTERVAL 6 month) and datetime_sub(first_episodestartdate,interval 5 month)),

#aggregate by antihypertensive type (based on bnf sub sub chapter)as only interested in whether the medication was prescribed once during peruiod
mpad_2 as(
select
person_id,
count(*), bnf_sub_sub_chapter, 
from mpad_1
where isrepeatmedication = 'true'
group by person_id,bnf_sub_sub_chapter
order by person_id),

mpad_array as(
select
person_id,
 ARRAY_AGG(DISTINCT BNF_sub_sub_chapter) AS antihypertensive
from mpad_2
group by person_id)

select
person_id,
IF('0206020' IN UNNEST(mpad_array.antihypertensive), 'true', 'false') AS calcium_channel_blocker,
IF('0204000' IN UNNEST(mpad_array.antihypertensive), 'true', 'false') AS beta_blocker,
IF('0202010' IN UNNEST(mpad_array.antihypertensive), 'true', 'false') AS thiazide_type_diuretic,
IF('0205052' IN UNNEST(mpad_array.antihypertensive),'true','false') as angiotensin_receptor_blocker,
IF('0205051' IN UNNEST(mpad_array.antihypertensive),'true','false') as ace_inhibitor,
IF('0205040' IN UNNEST(mpad_array.antihypertensive),'true','false') as alpha_blocker,
array_length(mpad_array.antihypertensive) as no_antihypertensive
from mpad_array
order by person_id)